# Caltech Machine Learning Homework # 2

In [1]:
import numpy as np
from sklearn.linear_model import Perceptron
import random
import math
import matplotlib.pyplot as plt
from typing import List
from itertools import product

Instructions: https://work.caltech.edu/homework/hw3.pdf

Answers: http://work.caltech.edu/homework/hw3_sol.pdf

## Generalization Error

![](imgs/generr1.png)

The bound should be

M = 1
epsilon = 0.05

2 * M * exp(-2 * (epsilon**2) * N) = 0.03

![](imgs/math1.jpg)

So the minimum number of N needed should be (b)

### 2. Repeat for the case M = 10

![](imgs/math2.jpg)

So

In [3]:
M = 10
N_bound = (np.log(0.03) - np.log(2) - np.log(M)) / (-2 * (0.05**2))
N_bound

1300.4580341747944

So the answer here is (c)

### 3. Repeat for the case M = 100

In [6]:
M = 100
N_bound = (np.log(0.03) - np.log(2) - np.log(M)) / (-2 * (0.05**2))
N_bound

1760.9750527736037

So the answer here is (d)

## Break Point

### 4.
As shown in class, the (smallest) break point for the Perceptron Model in the
two-dimensional case (R
2
) is 4 points. What is the smallest break point for
the Perceptron Model in R
3
? (i.e., instead of the hypothesis set consisting of
separating lines, it consists of separating planes.)

I would think 4 as well?

But it's wrong.

Why?

Let's build a 3D-Perceptron to find out. A smallest breakpoint `k` by definition means that there is a combination of `k-1` points that can be shattered by the 3D-Perceptron, but there is no combination of `k` points that can be shattered.

So let's first, for `N=4`, 

1. Generate `EXPERIMENTS` number of random samples of `N` points (x,y,z) (since that's the breakpoint for 2D perceptrons)
2. For each point, find all `2^N` dichotomies (possible solutions)
3. Attempt to fit all dichotomies, and see if 3D PLA converges for all of them. 
3a.) If it does, then we've "shattered" this data set of size `N`, and hence `N` cannot be a breakpoint.
3b.) If it doesn't, repeat to see if was just a "bad" random choice of set, and any of the others satisfy 3a). If not, we conclude that `N` is a breakpoint.

                                                


In [18]:
from sklearn.datasets import load_digits

X, y = load_digits(return_X_y=True)
print(X.shape)
print(y.shape)

(1797, 64)
(1797,)


In [56]:
N = 4
EXPERIMENTS = 1000

# Generate the set of all 2^N possible dichotomies (the cartesian product of [0,1] with itself, with N repetitions) 
DICHOTOMIES = np.array(list(product([0, 1], repeat = N)))

for _ in range(EXPERIMENTS):
    # Generate N random (x, y, z) points
    X = np.random.uniform(-1, 1, size=(N, 3))
    shattered = True
    for y in DICHOTOMIES:
        clf = Perceptron(tol=None) # We don't want the perceptron to converge at all if it can't find a solution
        
        # Working around this Perceptron model not allowing for only 1 class in the "training data"
        # That means we won't know if it shatters the points fully (2/16 solutions can't be generated)
        # But this should still do
        if len(np.unique(y)) == 1:
            clf.partial_fit(X, y, classes=np.array([0,1]))
        else:
            clf.fit(X, y) 
            
        if clf.score(X, y) != 1:
            print(f"Looks like we were unable to solve this data set for the dichotomy {y} on data set {X}")
            print(f"Score is {clf.score(X,y)}")
            shattered = False
            
    if shattered:
        print(f"We've shattered the data set {X}, hence {N} cannot be a breakpoint")
        break

We've shattered the data set [[ 0.30430459  0.68612827  0.13160432]
 [ 0.43895557  0.84705855  0.75776665]
 [ 0.64361878  0.71194008  0.65190386]
 [ 0.63154882 -0.10196242 -0.0353995 ]], hence 4 cannot be a breakpoint


We conclude that 4 cannot be a breakpoint. Let's try 5.

In [59]:
N = 5
EXPERIMENTS = 1000

# Generate the set of all 2^N possible dichotomies (the cartesian product of [0,1] with itself, with N repetitions) 
DICHOTOMIES = np.array(list(product([0, 1], repeat = N)))

for _ in range(EXPERIMENTS):
    # Generate N random (x, y, z) points
    X = np.random.uniform(-1, 1, size=(N, 3))
    shattered = True
    for y in DICHOTOMIES:
        clf = Perceptron(tol=None,shuffle=False) # We don't want the perceptron to converge at all if it can't find a solution
        
        # Working around this Perceptron model not allowing for only 1 class in the "training data"
        # That means we won't know if it shatters the points fully (2/16 solutions can't be generated)
        # But this should still do
        if len(np.unique(y)) == 1:
            clf.partial_fit(X, y, classes=np.array([0,1]))
        else:
            clf.fit(X, y) 
            
        if clf.score(X, y) != 1:
#             print(f"Looks like we were unable to solve this data set for the dichotomy {y} on data set {X}")
#             print(f"Score is {clf.score(X,y)}")
            shattered = False
            
    if shattered:
        print(f"We've shattered the data set {X}, hence {N} cannot be a breakpoint")
        break

We've tried 1000 different samples of size 5, and none could be shattered by the 3D Perceptron.

Hence we conclude that 5 is a breakpoint of the 3D Perceptron.